In [1]:
!pip install evaluate scikit-learn 'accelerate>=0.26.0'

# Import libraries

In [1]:
import evaluate
import torch
import numpy as np

from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

# Load Data

In [2]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

# Load Pre-trained Model

In [4]:
model_name = "google-bert/bert-base-uncased"
# load model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# loda model with binary classification head
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Set Trainable Parameters

In [5]:
# freaze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
# This will have the effect that we unfreeze the last 4 layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

## Data Pre-processing

In [6]:
def preprocess_data(examples):
    # return tokenized text with truncation
    return tokenizer(examples["text"], truncation=True)

# preprocess all datasets
tokenized_data = dataset_dict.map(preprocess_data, batched=True)

In [7]:
# create a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # Ensures all examples in a batch have uniform length

# Define evaluation metrics

In [14]:
from torch.nn.functional import softmax

accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # Apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)

    # use probabilities of the possitive class for ROC_AUC
    positive_class_probs = probabilities[:, 1]

    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'], 3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)

    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'], 3)

    return {"Accuracy": acc, "AUC": auc}

# Training parameters

In [15]:
lr = 2e-4
batch_size = 8
num_epochs = 20

training_args = TrainingArguments(
    output_dir = "bert-phishing-classifier-ktxdev",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Fine-tune Model

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/var/folders/z1/jkg9p4xd4lg8mfs514fzctfw0000gn/T/ipykernel_8238/1366999983.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.310100,0.387492,0.829000,0.942000
2,0.365600,0.350931,0.856000,0.945000
3,0.373200,0.295530,0.864000,0.947000
4,0.361400,0.425140,0.822000,0.947000
5,0.346100,0.290756,0.871000,0.950000
6,0.353500,0.325520,0.869000,0.952000
7,0.320500,0.282348,0.873000,0.952000
8,0.338200,0.317246,0.873000,0.951000
9,0.325000,0.316350,0.873000,0.956000
10,0.320500,0.355178,0.851000,0.957000


TrainOutput(global_step=5260, training_loss=0.3238444165131892, metrics={'train_runtime': 239.4468, 'train_samples_per_second': 175.404, 'train_steps_per_second': 21.967, 'total_flos': 1406151813158880.0, 'train_loss': 0.3238444165131892, 'epoch': 20.0})

# Validation Data

In [11]:
predictions = trainer.predict(tokenized_data['validation'])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.887), 'AUC': np.float64(0.949)}
